In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
from datetime import datetime, timedelta

In [71]:
def exrpesion(df, row): 
    unixTime = pd.Timestamp(0, unit='s')
    startHours = int((row['SubmitTime'] - unixTime).total_seconds()/3600)
    start = pd.to_datetime(startHours, unit='h')
    endHours = int((row['StopTime'] - unixTime).total_seconds()/3600)
    end = pd.to_datetime(endHours, unit='h')

    return (df['Timestamp'] >= start) & (df['Timestamp'] <= end)


def formCumulativeUsageDataset(path):

    #lambda date parser for year-month-day hour:minute:second
    dateParser = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S")
    dataset = pd.read_csv(path, delimiter=',', date_parser=dateParser, parse_dates=['SubmitTime', 'StopTime'])

    unixTime = pd.Timestamp(0, unit='s')

    minTime = dataset['SubmitTime'].min() - unixTime
    minHour = int(minTime.total_seconds()/3600)
    maxTime = dataset['StopTime'].max() - unixTime
    maxHour = int(maxTime.total_seconds()/3600)+1
    
    #create a new dataframe with the cumulative usage
    cumulativeUsage = pd.DataFrame(columns=['Hour', 'CPUTime', 'Memory', 'NJobs', 'NProcs'])
    cumulativeUsage['Timestamp'] = np.arange(minHour, maxHour)
    cumulativeUsage = cumulativeUsage.applymap(lambda x: pd.to_datetime(x, unit='h'))
    cumulativeUsage['NJobs'] = 0
    cumulativeUsage['CPUTime'] = 0
    cumulativeUsage['Memory'] = 0
    cumulativeUsage['NProcs'] = 0


    #calculate the cumulative usage for the 10 first rows

    for index, row in dataset.iterrows():
        cumulativeUsage.loc[lambda df: exrpesion(df, row), 'NJobs'] += 1
        cumulativeUsage.loc[lambda df: exrpesion(df, row), 'CPUTime'] += row['UsedCPUTime']
        cumulativeUsage.loc[lambda df: exrpesion(df, row), 'NProcs'] += row['NProc']
        cumulativeUsage.loc[lambda df: exrpesion(df, row), 'Memory'] += row['UsedMemory']

    
    cumulativeUsage.to_csv( f"./datasets/final/final-{path.split('/')[-1]}", index=False)

    return cumulativeUsage


In [69]:
df = formCumulativeUsageDataset('./datasets/formed/formed-auvergrid.csv')

/Users/alex/opt/anaconda3/lib/python3.9/site-packages/pandas/core/tools/datetimes.py:557: RuntimeWarning: invalid value encountered in cast
  arr, tz_parsed = tslib.array_with_unit_to_datetime(arg, unit, errors=errors)


In [12]:
df = pd.read_csv('./datasets/formed/formed-auvergrid.csv')
df.describe()

,JobID,WaitTime,RunTime,NProc,UsedCPUTime,UsedMemory,ReqTime,ReqMemory
count,404176.000000,4.041760e+05,4.041760e+05,404176.000000,404176.000000,4.041760e+05,404176.000000,4.041760e+05
mean,202088.500000,9.729772e+03,2.166128e+04,0.860049,18882.036120,2.542120e+05,221959.354675,7.631699e+04
std,116675.705537,5.140432e+04,3.881557e+04,0.346937,33011.668406,3.342076e+05,109882.318776,2.472621e+05
min,1.000000,0.000000e+00,-1.000000e+00,0.000000,-1.000000,-1.000000e+00,-1.000000,-1.000000e+00
25%,101044.750000,1.000000e+00,1.900000e+01,1.000000,3.000000,0.000000e+00,259200.000000,-1.000000e+00
50%,202088.500000,5.800000e+01,6.300000e+02,1.000000,21.000000,7.928000e+04,259200.000000,-1.000000e+00
75%,303132.250000,3.147000e+03,3.367800e+04,1.000000,30484.750000,5.750920e+05,259200.000000,-1.000000e+00
max,404176.000000,4.345394e+06,1.575814e+06,1.000000,259316.000000,3.667652e+06,345600.000000,1.126400e+06


In [13]:
df.corr()

/var/folders/rz/x38ry1m901jcjct47ydtrgph0000gn/T/ipykernel_54163/1134722465.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.corr()


,JobID,WaitTime,RunTime,NProc,UsedCPUTime,UsedMemory,ReqTime,ReqMemory
JobID,1.000000,-0.028045,-0.013272,0.044893,-0.013688,0.046346,0.073413,-0.020156
WaitTime,-0.028045,1.000000,0.082618,-0.009370,0.094106,0.037051,0.001670,0.028085
RunTime,-0.013272,0.082618,1.000000,0.225126,0.864234,0.419717,0.274379,-0.017943
NProc,0.044893,-0.009370,0.225126,1.000000,0.230745,0.306838,0.814846,0.124508
UsedCPUTime,-0.013688,0.094106,0.864234,0.230745,1.000000,0.447703,0.290692,-0.009307
UsedMemory,0.046346,0.037051,0.419717,0.306838,0.447703,1.000000,0.244945,0.175678
ReqTime,0.073413,0.001670,0.274379,0.814846,0.290692,0.244945,1.000000,-0.008853
ReqMemory,-0.020156,0.028085,-0.017943,0.124508,-0.009307,0.175678,-0.008853,1.000000
